Importing necessary libraries

In [150]:
# Import Libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from wordcloud import STOPWORDS
import nltk
import pycountry
from sklearn.feature_extraction.text import CountVectorizer
import random
import requests
import json
import nltk.sentiment
from nltk.sentiment import SentimentIntensityAnalyzer
import re

%matplotlib inline
%load_ext autoreload
%autoreload 2

# There's a lot of columns in the DF. 
# Therefore, we add this option so that we can see more columns
pd.options.display.max_columns = 100

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Sentiment analysis by country


Here we will find mentions of countries in Hillary Clinton's emails and try to assess the sentiment expressed by using sentiment analysis techniques.

In [2]:
emails = pd.read_csv('./hillary-clinton-emails/Emails.csv')
emails.head(2)

,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,MetadataDocumentClass,ExtractedSubject,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
0,1,C05739545,WOW,H,"Sullivan, Jacob J",87.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...,F-2015-04841,HRC_Email_296,FW: Wow,NaN,"Sullivan, Jacob J <Sullivan11@state.gov>",NaN,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,05/13/2015,RELEASE IN FULL,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
1,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,NaN,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,HRC_Email_296,NaN,NaN,NaN,NaN,NaN,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...


In [3]:
# concat all the emails into one string
from cleaner import *

# initiate Cleaner class
cleaner = Cleaner()
clean_text_series = cleaner.cleaning_pipeline_series(emails.ExtractedBodyText.dropna())

/Users/Jonas/anaconda/lib/python3.5/site-packages/bs4/__init__.py:282: UserWarning: "http://www.suntimes.com/2072121,desiree-rogers-quits-022610.article" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/Jonas/anaconda/lib/python3.5/site-packages/bs4/__init__.py:282: UserWarning: "http://www.brookings.edu/—/media/Files/rc/reports/2010/08_arab_opinion_poll_telhami/08_arab_opinion_pollielhami.pdf" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [4]:
df = pd.DataFrame({'Text':clean_text_series})
df.head()

,Text
1,"[thursday, march, 2011, latest, syria, aid, qa..."
2,[thx]
4,"[friday, march, 2011, huma, abedin, latest, sy..."
5,"[pi, print, wednesday, septemb, 2012, russorv,..."
7,"[friday, march, 2011, huma, abedin, latest, sy..."


## Creating a dictionnary of countries

We create a dictionnary containing different names for each country. We use an API to go and look for the country's capital city as well, as we believe analysing sentiment on the capital city is the same as on the country.

In [94]:
countries = {}

bullshit_countries = ["Antarctica"]

for country in pycountry.countries:
    if country.name not in bullshit_countries:
        list_to_search = []
        list_to_search.append(country.name.split(",")[0].lower())
        #list_to_search.append(country.alpha_2)
        #list_to_search.append(country.alpha_3)
        
        # getting the country's capital city's name
        try:
            str_text = requests.get("https://restcountries.eu/rest/v1/alpha/" + country.alpha_2.lower()).text
            dic = json.loads(str_text)
            list_to_search.append(dic["capital"].lower())
        except:
            print(str_text)
            pass

        # introducing some custom terms
        if country.name == "Syrian Arab Republic":
            list_to_search.append("syria")
        if country.name == "Russian Federation":
            list_to_search.append("russia")
        countries[country.name] = list_to_search

We introduce some custom gadgets

In [6]:
countries["UK"] = ['uk', 'united kingdom', 'great britain']
countries["United States"].append("U.S.")
countries["United States"].append("U.S.A.")

In [38]:
countries["Andorra"]

['andorra', 'AD', 'AND', 'andorra la vella']

In [14]:
emails.shape

(7945, 22)

In [52]:
bitch = '\n bitch \n biatch \n'
bitch.rfind('a')
x = bitch[:12].rfind('\n')
bitch[x+2:bitch[12:].find('\n')]

''

In [147]:
def isolate_line(s, n, marker):
    start = s[:n].rfind(marker) + len(marker)
    if start < 0:
        start = 0
    sub_s = s[start:]
    end = sub_s.find(marker)
    if end < 0:
        end = len(s)
    final = sub_s[:end]
    return final

In [148]:
bitches = 'avoineww. aseeeklas. bii'

In [149]:
isolate_line(bitches, bitches.find('bii'), '.')

' bii'

There is an issue with emails that mention multiple countries..

In [68]:
def demo_vader_instance(text):
    """
    Output polarity scores for a text using Vader approach.

    @param text: a text whose polarity has to be evaluated.
    @return vader_analyszer : the result of the analysis with Vader approach
    """
    vader_analyzer = SentimentIntensityAnalyzer()
    #print(vader_analyzer.polarity_scores(text))
    return vader_analyzer

In [78]:
len(emails)

7945

0
1
2


In [ ]:
stopwords = list(STOPWORDS)
test = ''
cited_countries = {}
for i in range(0, len(emails)):
    print(i)
    body = str(emails.iloc[i].ExtractedBodyText)
    body_lower = body.lower()
    subject = str(emails.iloc[i].ExtractedSubject).lower()
    
    body_array = body.split(" ")
    #print('len1: '+str(len(body)))
    body_array.extend(body_lower.split(" "))
    #print('len2: '+str(len(body)))
    
    subject = subject.split(" ")
    if body_array != "nan":  
        for ctr in countries:
            for mention in countries[ctr]:
                
                #if (mention in body_array or mention in subject):
                if (mention in body_array):
                    print('m: '+mention)
                    mentions = [m.start() for m in re.finditer(mention, body+body_lower)]
                    print('mentions: '+str(mentions))
                    for start in mentions:
                        sentence = isolate_line(body+body_lower, start, '.')
                        v = demo_vader_instance(sentence)
                        senti = v.polarity_scores(sentence)


                        if ctr in cited_countries:
                            cited_countries[ctr]["nbr_mention"] += 1
                            cited_countries[ctr]["mention"].append(mention)
                            cited_countries[ctr]['pos'].append(senti['pos'])
                            cited_countries[ctr]['neg'].append(senti['neg'])
                        else:

                            subdic = {}
                            subdic["pos"] = [senti['pos']]
                            subdic['neg'] = [senti['neg']]
                            subdic["average_sentiment"] = 0
                            subdic["nbr_mention"] = 1
                            subdic["mention"] = [mention]
                            cited_countries[ctr] = subdic
                            subdic["text"] = body
                            subdic['sentence'] = sentence
                            test = body
                        
                        print(sentence)
                        print(v.polarity_scores(sentence))
                        print(' ')
                    break
    

0
1
m: libya
mentions: [111, 152, 307, 348]
 Sid
hrc memo syria aiding libya 030311
{'pos': 0.0, 'neu': 1.0, 'compound': 0.0, 'neg': 0.0}
 
docx; hrc memo syria aiding libya 030311
{'pos': 0.0, 'neu': 1.0, 'compound': 0.0, 'neg': 0.0}
 
 sid
hrc memo syria aiding libya 030311
{'pos': 0.0, 'neu': 1.0, 'compound': 0.0, 'neg': 0.0}
 
docx; hrc memo syria aiding libya 030311
{'pos': 0.0, 'neu': 1.0, 'compound': 0.0, 'neg': 0.0}
 
m: syria
mentions: [98, 139, 245, 294, 335]
 Sid
hrc memo syria aiding libya 030311
{'pos': 0.0, 'neu': 1.0, 'compound': 0.0, 'neg': 0.0}
 
docx; hrc memo syria aiding libya 030311
{'pos': 0.0, 'neu': 1.0, 'compound': 0.0, 'neg': 0.0}
 
docx
March 3, 2011
For: Hillaryb6
thursday, march 3, 2011 9:45 pm
h: latest how syria is aiding qaddafi and more
{'pos': 0.0, 'neu': 1.0, 'compound': 0.0, 'neg': 0.0}
 
 sid
hrc memo syria aiding libya 030311
{'pos': 0.0, 'neu': 1.0, 'compound': 0.0, 'neg': 0.0}
 
docx; hrc memo syria aiding libya 030311
{'pos': 0.0, 'neu': 1.0, 'c

In [127]:
emails.iloc[214].ExtractedBodyText

'Attached is full compilation. You never said spontaneous or characterized the motives. in fact you were careful in your\nfirst statement to say we were assessing motive and method. The way you treated the video in the Libya context was to\nsay that some sought to *justify* the attack on that basis.\nFrom Valliani, Amira\nSent: Monday, September 24, 2012 10:08 AM\nTo: Sullivan, Jacob 3\nSubject: Berthazi/protest statements\nAttached\nU.S. Department of State\nCase No. F-2015-04841\nDoc No. C05739812\nDate: 05/13/2015\nSTATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM.\nSUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-SCB0045474\nU.S. Department of State\nCase No. F-2015-04841\nDoc No. C05739812\nDate: 05/13/2015\nSTATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM.\nSUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER.\nStatement on the Attack in Benghazi\nSeptember 11, 2012\nI condemn in the strongest terms the attack on our m

In [158]:
mainframe = pd.DataFrame(columns=['country', 'n_mention', 'pos', 'neg'])
for key, value in cited_countries.items():
    print(key)
    df = make_frame(key, value)
    print(df)
    mainframe = mainframe.append(df, ignore_index=True)

Azerbaijan
      country  n_mention     pos    neg
0  Azerbaijan          2  0.1175  0.045
Libya
  country  n_mention       pos       neg
0   Libya         88  0.061432  0.104443
Malta
  country  n_mention  pos  neg
0   Malta          1  0.0  0.0
Iran, Islamic Republic of
                     country  n_mention       pos       neg
0  Iran, Islamic Republic of          7  0.086714  0.087571
Chad
  country  n_mention  pos    neg
0    Chad          3  0.0  0.059
Israel
  country  n_mention       pos       neg
0  Israel          9  0.095111  0.051111
Tunisia
   country  n_mention    pos    neg
0  Tunisia          1  0.116  0.099
Norway
  country  n_mention    pos  neg
0  Norway          1  0.136  0.0
Morocco
   country  n_mention  pos    neg
0  Morocco          1  0.0  0.069
Qatar
  country  n_mention      pos     neg
0   Qatar          4  0.04225  0.0545
Seychelles
      country  n_mention  pos    neg
0  Seychelles          1  0.0  0.135
Yemen
  country  n_mention  pos  neg
0   Yemen     

In [159]:
mainframe

,country,n_mention,pos,neg
0,Azerbaijan,2.0,0.117500,0.045000
1,Libya,88.0,0.061432,0.104443
2,Malta,1.0,0.000000,0.000000
3,"Iran, Islamic Republic of",7.0,0.086714,0.087571
4,Chad,3.0,0.000000,0.059000
5,Israel,9.0,0.095111,0.051111
6,Tunisia,1.0,0.116000,0.099000
7,Norway,1.0,0.136000,0.000000
8,Morocco,1.0,0.000000,0.069000
9,Qatar,4.0,0.042250,0.054500


In [116]:
def make_frame(key, value):
    n_mention = int(len(value['mention']))
    pos = sum(value['pos'])/float(len(value['pos']))
    neg = sum(value['neg'])/float(len(value['neg']))
    df = pd.DataFrame([[key, n_mention, pos, neg]], columns=['country', 'n_mention', 'pos', 'neg'])
    return df

In [106]:
bitch

,bitch,bitches
0,1,2


,bitch,bitches,0
0,NaN,NaN,5.0
1,NaN,NaN,6.0


In [20]:
cited_countries

{'Andorra': {'average_sentiment': 0,
  'citation': ['AND', 'AND'],
  'nbr_citation': 2,
  'sentiment': [],
  'text': ['8.25',
   'am',
   'DEPART',
   'Private',
   'Residence',
   'En',
   'route',
   'State',
   'Department\n8:35',
   'am',
   'ARRIVE',
   'State',
   'Department\n8,35',
   'am',
   'PRESIDENTIAL',
   'DAILY',
   'BRIEFING\n8.40',
   'am',
   "Secretary's",
   'Office\n8:45',
   'am',
   'DAILY',
   'SENIOR',
   'STAFF',
   'MEETING\n9:15',
   'am',
   "Secretary's",
   'Conference',
   'Room\n9:20',
   'am',
   'GROUP',
   'PHOTO',
   'wfFEMALlE',
   'MILITARY',
   'OFFICERS',
   'FROM',
   'AFRICA\n9:25',
   'am',
   'Treaty',
   'Ksom,',
   '7',
   'Floor\nClosed',
   'Press',
   '(Official',
   'Photographer',
   'Only),',
   'Staff:',
   'Monica',
   ',',
   'WS',
   'Carson',
   'will',
   'greet',
   'and',
   'escort',
   'to',
   'the',
   'Treaty',
   'Room\n-Approx.',
   '25-30',
   'people',
   'attending;',
   'consecutive',
   'French',
   'and',
   'Po

In [22]:
test

['-',
 'do',
 'you',
 'approve',
 'the',
 'War',
 'Powers',
 'Act',
 'action',
 'memo?\n-',
 'The',
 'King',
 'of',
 'Morocco',
 'has',
 'not',
 "gotten'",
 'back',
 'to',
 'us',
 'with',
 'a',
 'time',
 'but',
 'ops',
 'will',
 'try',
 'again',
 'when',
 'its',
 'a',
 'reasonable',
 'hour',
 'for',
 'the\nMoroccans.\n-',
 'Attorney',
 'General',
 'Holder',
 'has',
 'requested',
 'to',
 'speak',
 'with',
 'you',
 'regarding',
 'the',
 'Department',
 'of',
 "Justice's",
 'Libya',
 'investigation.',
 'He\nis',
 'standing',
 'by',
 'Would',
 'you',
 'like',
 'to',
 'take',
 'this',
 'tonight?\nU.S.',
 'Department',
 'of',
 'State\nCase',
 'No.',
 'F-2015-04841\nDoc',
 'No.',
 'C05739662\nDate:',
 '05/13/2015\nSTATE',
 'DEPT.',
 '-',
 'PRODUCED',
 'TO',
 'HOUSE',
 'SELECT',
 'BENGHAZI',
 'COMM.\nSUBJECT',
 'TO',
 'AGREEMENT',
 'ON',
 'SENSITIVE',
 'INFORMATION',
 '&',
 'REDACTIONS.',
 'NO',
 'FOIA',
 'WAIVER.',
 'STATE-SCB0045339',
 '-',
 'do',
 'you',
 'approve',
 'the',
 'war',
 'powers'

In [29]:
nltk.sentiment.util.demo_liu_hu_lexicon(sentence, plot=False)

Positive


In [26]:
sentence = ''
for t in test:
    sentence += str(t)+' '

In [27]:
sentence

"- do you approve the War Powers Act action memo?\n- The King of Morocco has not gotten' back to us with a time but ops will try again when its a reasonable hour for the\nMoroccans.\n- Attorney General Holder has requested to speak with you regarding the Department of Justice's Libya investigation. He\nis standing by Would you like to take this tonight?\nU.S. Department of State\nCase No. F-2015-04841\nDoc No. C05739662\nDate: 05/13/2015\nSTATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM.\nSUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-SCB0045339 - do you approve the war powers act action memo?\n- the king of morocco has not gotten' back to us with a time but ops will try again when its a reasonable hour for the\nmoroccans.\n- attorney general holder has requested to speak with you regarding the department of justice's libya investigation. he\nis standing by would you like to take this tonight?\nu.s. department of state\ncase no. f-2015-04841\ndoc 

In [43]:
bitch = '\n bitch \n biatch \n'
bitch.rfind('a')

12

In [62]:
biatch = 'STATE-5CB0045603'
biatch.isupper()

True